## Problem 3

Let's import all images from the 6 categories and reshape them to (64,64,3). We will make one  array containing all images and another housing their labels.

### Data Preprocess

In [2]:
from os import listdir
from skimage.io import imread
from skimage.color import gray2rgb
from skimage.transform import resize
import numpy as np

Due to having a very limited amount of computational power, I elect to resize each image to a size of (64,64,3) so that computations become tractable and displayable on this notebook.

In [3]:
def getImages(paths, alllabels):
    images, labels = [],[]
    for p in range(len(paths)):
        imagesList = listdir(paths[p])
        for image in imagesList:
            im = imread(paths[p] + image)
            if len(im.shape) == 2:
                im = gray2rgb(im)
            im = resize(im, (64,64,3), preserve_range=False)
            images.append(im)
            labels.append(alllabels[p])
    return images, labels

In [4]:
path = '/Users/Karel/cs/cse6240/hw2/'
imlabels = ['person','animal','plant','sport','geology','fungus']
images, labels = getImages([path+'n00007846/',path+'n00015388/',\
                    path+'n00017222/',path+'n00523513/',path+'n09287968/',path+'n12992868/'],\
                          imlabels)

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.20)

In [14]:
from keras.callbacks import TensorBoard
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

In [6]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

### Autoencoder

In [32]:
input_img = Input(shape=(64, 64, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)


x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['mse'])

In [35]:
autoencoder.fit(x_train, x_train,
                epochs=3,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Train on 7189 samples, validate on 1798 samples
Epoch 1/3
7189/7189 [==============================] - 770s - loss: 0.6597 - mean_squared_error: 0.0624 - val_loss: 0.6205 - val_mean_squared_error: 0.0441
Epoch 2/3
7189/7189 [==============================] - 524s - loss: 0.5985 - mean_squared_error: 0.0351 - val_loss: 0.5887 - val_mean_squared_error: 0.0304
Epoch 3/3
7189/7189 [==============================] - 507s - loss: 0.5864 - mean_squared_error: 0.0301 - val_loss: 0.5796 - val_mean_squared_error: 0.0273


In [39]:
encoder = Model(input_img,encoded)

In [40]:
train_encoded = encoder.predict(x_train)
test_encoded = encoder.predict(x_test)

In [57]:
train_encoded1 = train_encoded.reshape(len(train_encoded),16*16*32)
print train_encoded1.shape

(7189, 8192)


In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [58]:
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(train_encoded1, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [60]:
test_encoded1 = test_encoded.reshape(len(test_encoded),16*16*32)

In [61]:
x_pred = knn.predict(test_encoded1)

In [16]:
def makeConfusion(xpred,xtrue):
    return confusion_matrix(xtrue, xpred)

We can now compute the autoencoder confusion matrix using euclidean distance for kNN and its corresponding accuracy.

In [86]:
confusionMatrix = makeConfusion(x_pred,y_test)
print confusionMatrix
print '\nAccuracy: %f' %accuracy_score(x_pred,y_test)

[[115  26 134   6  39  14]
 [ 32  84  55   2  57   4]
 [ 46   9 266   4  17  11]
 [ 67  18 103  36  26   7]
 [ 41  33  58   1 103   2]
 [ 94  18 176   9  33  52]]

Accuracy: 0.364850


Now we perform the same analysis but using the Pearson correlation coefficient as our similarity score.

In [89]:
knn_P = KNeighborsClassifier(n_neighbors=5, metric='correlation',algorithm='brute')
knn_P.fit(train_encoded1, y_train)
x_pred_P = knn_P.predict(test_encoded1)
confusionMatrix_P = makeConfusion(x_pred_P,y_test)
print confusionMatrix_P
print '\nAccuracy: %f' %accuracy_score(x_pred_P,y_test)

[[110  28 110  20  50  16]
 [ 43  82  55   5  47   2]
 [ 43  11 265   5  22   7]
 [ 57  26 101  43  19  11]
 [ 33  37  62   8  96   2]
 [ 88  20 182   9  29  54]]

Accuracy: 0.361513


### RGB Histogram

In [7]:
def getRgbColumns(imageMatrix):
    r, g, b = [],[],[]
    for row in imageMatrix:
        for col in row:
            r.append(col[0])
            g.append(col[1])
            b.append(col[2])
    return r,g,b

In [116]:
def getBins(arr):
    return np.histogram(arr, bins=np.arange(0,1+float(1)/256,float(1)/256))[0]

In [8]:
def makeFV(arr1,arr2,arr3):
    return np.concatenate((arr1,arr2,arr3))

In [121]:
def makeAllFV(samplesMatrix):
    fvs = []
    for sample in samplesMatrix:
        r,g,b = getRgbColumns(sample)
        rbin,gbin,bbin = getBins(r), getBins(g), getBins(b)
        fvs.append(makeFV(rbin,gbin,bbin))
    return fvs

In [124]:
x_train_rgb = makeAllFV(x_train)

In [125]:
x_test_rgb = makeAllFV(x_test)

Now we will use kNN with euclidean distance metric to generate the confusion matrix and accuracy results.

In [126]:
knn_RGB = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn_RGB.fit(x_train_rgb, y_train)
x_pred_RGB = knn_RGB.predict(x_test_rgb)
confusionMatrix_RGB = makeConfusion(x_pred_RGB,y_test)
print confusionMatrix_RGB
print '\nAccuracy: %f' %accuracy_score(x_pred_RGB,y_test)

[[112  88  52  29  45   8]
 [ 26 151   8  14  22  13]
 [ 86  82 147  16  12  10]
 [ 62  73  30  54  16  22]
 [ 33  71  32   8  87   7]
 [ 93  90  51  28  19 101]]

Accuracy: 0.362625


Next we will do the same process, but using the Pearson correlation coefficient to rank closeness of vectors.

In [128]:
knn_RGB_P = KNeighborsClassifier(n_neighbors=5, metric='correlation',algorithm='brute')
knn_RGB_P.fit(x_train_rgb, y_train)
x_pred_RGB_P = knn_RGB_P.predict(x_test_rgb)
confusionMatrix_RGB_P = makeConfusion(x_pred_RGB_P,y_test)
print confusionMatrix_RGB_P
print '\nAccuracy: %f' %accuracy_score(x_pred_RGB_P,y_test)

[[ 97  59  59  38  61  20]
 [ 30 131  17  16  34   6]
 [ 73  44 183  21  23   9]
 [ 53  55  39  68  22  20]
 [ 36  50  33  13 102   4]
 [ 98  45  78  32  28 101]]

Accuracy: 0.379310


### HSV Histogram

First, lets convert our data from rgb to hsv.

In [9]:
from skimage import color
x_train_hsv, x_test_hsv = [],[]
for image in x_train:
    x_train_hsv.append(color.rgb2hsv(image))
for image in x_test:
    x_test_hsv.append(color.rgb2hsv(image))

We will reuse some of the functions used in RGB Histogram, but slightly alter others to fit the needs of the hsv representation.

In [10]:
def getBins2(arr, numBins):
    return np.histogram(arr, bins=np.arange(0,1+float(1)/numBins,float(1)/numBins))[0]

In [11]:
def makeAllFV2(samplesMatrix):
    fvs = []
    for sample in samplesMatrix:
        h,s,v = getRgbColumns(sample)
        hbin,sbin,vbin = getBins2(h,180), getBins2(s,256), getBins2(v,256)
        fvs.append(makeFV(hbin,sbin,vbin))
    return fvs

Now we can make the feature vector representation of our hsv data.

In [12]:
x_train_hsv1 = makeAllFV2(x_train_hsv)
x_test_hsv1 = makeAllFV2(x_test_hsv)

Now we will use kNN with euclidean distance metric to generate the confusion matrix and accuracy results.

In [17]:
knn_hsv = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn_hsv.fit(x_train_hsv1, y_train)
x_pred_hsv1 = knn_hsv.predict(x_test_hsv1)
confusionMatrix_hsv = makeConfusion(x_pred_hsv1,y_test)
print confusionMatrix_hsv
print '\nAccuracy: %f' %accuracy_score(x_pred_hsv1,y_test)

[[ 87 100  53  16  40  10]
 [ 22 151  13   6  23   9]
 [ 61  80 192  17  15  25]
 [ 41  65  35  63  12  22]
 [ 43  70  20   6 129   6]
 [ 61 109  34  12  23 127]]

Accuracy: 0.416574


We will repeat this process with the Pearson correlation coefficient as the similarity metric.

In [18]:
knn_hsv_P = KNeighborsClassifier(n_neighbors=5, metric='correlation',algorithm='brute')
knn_hsv_P.fit(x_train_hsv1, y_train)
x_pred_hsv_P = knn_hsv_P.predict(x_test_hsv1)
confusionMatrix_hsv_P = makeConfusion(x_pred_hsv_P,y_test)
print confusionMatrix_hsv_P
print '\nAccuracy: %f' %accuracy_score(x_pred_hsv_P,y_test)

[[112  58  68  20  40   8]
 [ 46 112  29   6  24   7]
 [ 71  49 229  24   9   8]
 [ 40  52  45  72   9  20]
 [ 52  54  22  11 126   9]
 [ 66  73  70  18  30 109]]

Accuracy: 0.422692


### SVD

First we will flatten out data so each image is a one dimensional vector.

In [19]:
x_train_svd = []
x_test_svd = []
for image in x_train:
    x_train_svd.append(np.reshape(image,64*64*3))
for image in x_test:
    x_test_svd.append(np.reshape(image,64*64*3))

Now we can perform Singular Value Decomposition on the flattened vectors. Given the large number of features in each image, I elected to choose a k value of several hundred so as not to lose too much information from the original image's high dimensionality.

In [23]:
from sklearn.decomposition import TruncatedSVD

In [24]:
svd = TruncatedSVD(n_components=256)
x_train_svd1 = svd.fit_transform(x_train_svd)
x_test_svd1 = svd.transform(x_test_svd)

We can now use these vectors and analyze its accuracy using 5-NN with euclidean distance as the distance metric.

In [25]:
knn_svd = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn_svd.fit(x_train_svd1, y_train)
x_pred_svd1 = knn_svd.predict(x_test_svd1)
confusionMatrix_svd = makeConfusion(x_pred_svd1,y_test)
print confusionMatrix_svd
print '\nAccuracy: %f' %accuracy_score(x_pred_svd1,y_test)

[[114  30 110   7  23  22]
 [ 69  58  69   3  21   4]
 [ 44  10 306   4  13  13]
 [ 54  23 100  36  12  13]
 [ 68  22  83   4  85  12]
 [106   9 159   5  22  65]]

Accuracy: 0.369299


Similarly, using 5-NN and the Pearson correlation coefficient.

In [26]:
knn_svd_P = KNeighborsClassifier(n_neighbors=5, metric='correlation',algorithm='brute')
knn_svd_P.fit(x_train_svd1, y_train)
x_pred_svd_P = knn_svd_P.predict(x_test_svd1)
confusionMatrix_svd_P = makeConfusion(x_pred_svd_P,y_test)
print confusionMatrix_svd_P
print '\nAccuracy: %f' %accuracy_score(x_pred_svd_P,y_test)

[[118  14 116  22  15  21]
 [ 62  53  77   4  24   4]
 [ 38   6 314  12   6  14]
 [ 55  15 111  32  12  13]
 [ 71  20  94   3  81   5]
 [ 94  15 166  17  12  62]]

Accuracy: 0.367075
